<a href="https://colab.research.google.com/github/jpbeaud/dbscan/blob/main/dbscanv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import time

def string_to_number(s):
  return int(''.join(str(ord(char)) for char in s))



chemin = "/content/sample_data/"
debut = time.time()
print("Début à", debut)



fichier_DH = pd.read_csv(chemin+"extrac_DH.csv", sep=",", dtype=str,skipinitialspace=True)

fichier_DH = fichier_DH.replace("", np.nan)
fichier_DH = fichier_DH.fillna(0)

fichier_DH = fichier_DH.astype(str)
col_ref = ['id_personne']
references_DH = fichier_DH[col_ref]



fichier_DH.sort_values(by='id_personne')

fichier_DH2 = fichier_DH
fichier_DH2 = fichier_DH2.drop('id_personne', axis='columns')

fichier_DH2= fichier_DH2.astype(str)
fichier_DH2 = fichier_DH2.map(string_to_number)
print ("fichier DH2 format :")
print(fichier_DH2.head())


#fichier_DH.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF




fichier_SIO = pd.read_csv(chemin+"extrac_SIO.csv", sep=",", dtype=str,skipinitialspace=True)
col_ref = ['client_reference']

references_SIO= fichier_SIO[col_ref]
ref_cli = references_SIO.to_numpy()

fichier_SIO = fichier_SIO.replace("", np.nan)
fichier_SIO = fichier_SIO.fillna(0)



fichier_SIO.sort_values(by='client_reference')
fichier_SIO = fichier_SIO.astype(str)
fichier_SIO = fichier_SIO.map(string_to_number)


fichier_SIO = fichier_SIO.drop('client_reference',axis='columns')
# X = fichier_DH.to_numpy()
X = fichier_DH2
#X.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF

# print(X.dtype)
X = X.apply(pd.to_numeric)

import matplotlib.pyplot as plt
# pltcatter(X[:, 3], X[:, 4])


#plt.scatter(x="pcs" , y="cosop", c = 'r', data=X)
#plt.show()

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# création du jeu de test

#X = StandardScaler().fit_transform(X)

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
min_groupe = 1
db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
labels_DH = db.labels_

n_clusters_ = len(set(labels_DH)) - (1 if -1 in labels_DH else 0)
n_noise_ = list(labels_DH).count(-1)

print("Estimated number of clusters DH: %d" % n_clusters_)
print("Estimated number of noise points DH: %d" % n_noise_)


Y = fichier_SIO
Y = Y.apply(pd.to_numeric)
db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
labels_SIO = db2.labels_



n_clusters_ = len(set(labels_SIO)) - (1 if -1 in labels_SIO else 0)
n_noise_ = list(labels_SIO).count(-1)

print("Estimated number of clusters SIO: %d" % n_clusters_)
print("Estimated number of noise points SIO: %d" % n_noise_)
print("____________Résumé____________")
difference = labels_DH - labels_SIO
print("Données en écarts = ", difference)
print(labels_DH)
print(labels_SIO)
n_DH = len(labels_DH)
n_SIO=len(labels_SIO)
n_dif = len(difference)
print("Longueur dif = ", n_dif)
fin = False
i=-1

nb_ko = 0
while (i < n_dif - 1):
    i = i + 1
    if (difference[i] != 0):
        print("i=",i)
        print("différence enregistrement ", str(i), " DH=", labels_DH[i], "// SIO=",labels_SIO[i], "// ref=", ref_cli[i,:])
        #print("enr DH=", fichier_DH.loc[i,:])
        #print("enr SIO=",fichier_SIO.loc[i,:])
        nb_ko = nb_ko + 1
        X = np.delete(X,i,0)
        Y = np.delete(Y,i,0)
        ref_cli = np.delete(ref_cli,i,0)
        db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
        labels_DH = db.labels_
        db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
        labels_SIO = db2.labels_
        difference = labels_DH - labels_SIO
        n_dif = n_dif - 1


lg = len(difference)
#nb_ano = 0
#for i in range(lg):
#    if difference[i] !=0:
#        nb_ano = nb_ano + 1
print("ANOMALIES = ", nb_ko)
#  3117773703

fin_exec = time.time()
temps = fin_exec - debut
print(f'Temps d\'exécution : {temps:.2}ms')

Début à 1730291275.2382913
id_personne                object
code_dept_naiss            object
code_lim_jur               object
code_pays_naiss            object
code_pays_nat_1            object
code_sit_familiale         object
intitule_lieu_naissance    object
dtype: object
fichier DH2 format :
   code_dept_naiss  code_lim_jur  code_pays_naiss  code_pays_nat_1  \
0             5253            48             7082             7082   
1             5351            48             7082             7082   
2             5354            48             7082             7082   
3             5253            48             7082             7082   
4             5056            48             7082             7082   

   code_sit_familiale               intitule_lieu_naissance  
0                4850                        79827669657883  
1                4850  867376766573786983327665327485726976  
2                4849                    726978786966797884  
3                4849          

KeyboardInterrupt: 

;l;

Lecture fichier DH